# Inference Notebook

Thanks to the public 11th place solution Inference Notebook: https://www.kaggle.com/code/shigemitsutomizawa/shopee-inference-11th-place-simple-solution

In [ ]:
import sys
if 'kaggle_web_client' in sys.modules:
    sys.path.append('../input/imports/pytorch-image-models-master/pytorch-image-models-master')
    sys.path.append('../input/imports/transformers-master/transformers-master')

In [ ]:
import os
import re
import cv2
import math
import random
import numpy as np
import pandas as pd
import gc
import matplotlib.pyplot as plt
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
from torch.nn import functional as F
from torch.utils.data import Dataset, DataLoader
import timm
import albumentations
from albumentations.pytorch.transforms import ToTensorV2

from transformers import AutoTokenizer, AutoModel

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.neighbors import NearestNeighbors

## Config

In [ ]:
class CFG:
    compute_cv = 0  # set False to fast save
    todo_predictions = ['predictions']
    
    ### CNN and cnn
    use_amp = True
    scale = 30  # ArcFace
    margin = 0.5  # ArcFace
    seed = 2021
    classes = 11014
    device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
    print(device)           
    
    ### Prediction
    cnn_threshold = 0.68
    chunk = 1024
    max_preds = 42
    nearlest_one = False # True is better
        
    ### Data
    
    train_csv_path = '../input/shopee-product-matching/train.csv'
    test_csv_path = '../input/shopee-product-matching/test.csv'
    
    if compute_cv == True:
        images_dir = '../input/shopee-product-matching/train_images/'
    else:
        images_dir = '../input/shopee-product-matching/test_images/'

    if 'kaggle_web_client' in sys.modules:
        num_workers = 4
    else:
        num_workers = 0  # for Windows 10

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True # set True to be faster

seed_everything(CFG.seed)

## Utils

In [ ]:
def read_dataset():
    
    df = pd.read_csv(CFG.test_csv_path)
    
    if len(df) > 3:
        CFG.compute_cv = False
        CFG.images_dir = '../input/shopee-product-matching/test_images/'
    
    if CFG.compute_cv == True:
        df = pd.read_csv(CFG.train_csv_path)
        print('Using train as test to compute CV. Shape is', df.shape)
    else:
        print('Test shape is', df.shape )
    
    image_paths = CFG.images_dir + df['image']

    return df, image_paths

In [ ]:
def f1_score(y_true, y_pred):
    y_true = y_true.apply(lambda x: set(x.split()))
    y_pred = y_pred.apply(lambda x: set(x.split()))
    intersection = np.array([len(x[0] & x[1]) for x in zip(y_true, y_pred)])
    len_y_pred = y_pred.apply(lambda x: len(x)).values
    len_y_true = y_true.apply(lambda x: len(x)).values
    f1 = 2 * intersection / (len_y_pred + len_y_true)
    return f1

In [ ]:
def get_cnn_embeddings(df, column, model, fc_dim=768, chunk=32):
    
    print('Getting embeddings...')
    
    cnn_embeddings = torch.zeros((df.shape[0], fc_dim)).to(CFG.device)
    for i in tqdm(list(range(0, df.shape[0], chunk)) + [df.shape[0]-chunk], ncols=100):
        titles = []
        for title in df[column][i : i + chunk].values:
            try:
                title = ' ' + title.encode('utf-8').decode("unicode_escape").encode('ascii', 'ignore').decode("unicode_escape") + ' '
            except:
                pass
            title = title.lower()
            
            titles.append(title)
            
        with torch.no_grad():
            if CFG.use_amp:
                with torch.cuda.amp.autocast():
                    model_output = model.extract_features(titles)
            else:
                model_output = model.extract_features(titles)
            
        cnn_embeddings[i : i + chunk] = model_output
    
    del model, titles, model_output
    gc.collect()
    torch.cuda.empty_cache()
    
    return cnn_embeddings

## Image Model

In [ ]:
def gem(x, p=3, eps=1e-6):
    return F.avg_pool2d(x.clamp(min=eps).pow(p), (x.size(-2), x.size(-1))).pow(1./p)

    
class ShopeeNet(nn.Module):

    def __init__(self,
                 backbone,
                 num_classes,
                 fc_dim=512,
                 s=30, margin=0.5, p=3):
        super(ShopeeNet, self).__init__()

        self.backbone = backbone
        self.backbone.reset_classifier(num_classes=0)  # remove classifier

        self.classifier = nn.Linear(self.backbone.num_features, fc_dim)
        self.bn = nn.BatchNorm1d(fc_dim)
        self._init_params()
        self.p = p

    def _init_params(self):
        nn.init.xavier_normal_(self.classifier.weight)
        nn.init.constant_(self.classifier.bias, 0)
        nn.init.constant_(self.bn.weight, 1)
        nn.init.constant_(self.bn.bias, 0)

    def extract_feat(self, x):
        batch_size = x.shape[0]
        x = self.backbone.forward_features(x)
        if isinstance(x, tuple):
            x = (x[0] + x[1]) / 2
            x = self.bn(x)
        else:
            x = gem(x, p=self.p).view(batch_size, -1)
            x = self.classifier(x)
            x = self.bn(x)
        return x


In [ ]:
class ShopeeDataset(Dataset):

    def __init__(self, df, img_dir, transform=None):
        self.df = df
        self.img_dir = img_dir
        self.transform = transform

    def __getitem__(self, index):
        row = self.df.iloc[index]
        img = read_image(str(self.img_dir + row['image']))
        if self.transform is not None:
            img = self.transform(img.float() / 255)
        return img

    def __len__(self):
        return len(self.df)

In [ ]:
from PIL import Image
from torchvision.io import read_image
from torchvision.transforms import Resize, RandomHorizontalFlip, ColorJitter, Normalize, Compose, RandomResizedCrop, CenterCrop, ToTensor

In [ ]:

params1 = {'ver': 'v45', 'size': 384, 'test_size': 384, 'lr': 0.001, 'batch_size': 32, 'optimizer': 'sam', 'epochs': 18, 'wd': 0.0, 'backbone': 'vit_deit_base_distilled_patch16_384', 'margin': 0.3, 's': 50, 'fc_dim': 768, 'brightness': 0.2, 'contrast': 0.2, 'scale_lower': 0.2, 'scale_upper': 1.0, 'filter_wd': True, 'p': 3.0, 'p_eval': 6.0, 'loss': 'CurricularFace'}
params2 = {'ver': 'v34', 'size': 256, 'test_size': 320, 'lr': 0.001, 'batch_size': 32, 'optimizer': 'sam', 'epochs': 11, 'wd': 0.0, 'backbone': 'dm_nfnet_f0', 'margin': 0.3, 's': 50, 'fc_dim': 256, 'brightness': 0.2, 'contrast': 0.2, 'scale_lower': 0.2, 'scale_upper': 1.0, 'filter_wd': True, 'p': 3.0, 'p_eval': 6.0, 'loss': 'CurricularFace'}
params3 = {'ver': 'v79', 'lr': 0.001, 'batch_size': 16, 'size': 256, 'test_size': 320, 'optimizer': 'sam', 'epochs': 8, 'loss': 'CurricularFace', 'wd': 1e-05, 'filter_wd': True, 'margin': 0.3, 's': 50, 'fc_dim': 1024, 'cycle': 1, 'backbone': 'dm_nfnet_f0', 'model_name': 'cahya/cnn-base-indonesian-522M', 'max_len': 64, 'brightness': 0.2, 'contrast': 0.2, 'scale_lower': 0.2, 'scale_upper': 1.0, 'p': 3.0, 'p_eval': 6.0}

In [ ]:
transform = Compose([
    Resize(size=params1['test_size'] + 32, interpolation=Image.BICUBIC),
    CenterCrop((params1['test_size'], params1['test_size'])),
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [ ]:
def get_cnn_embeddings(model, dataloader):
    model.eval()

    embeds = []
    for _, image in tqdm(enumerate(dataloader), total=len(dataloader), desc="get_cnn_embeddings", ncols=80): 
#         print(type(image), image.shape)
        img = image.to(CFG.device)

        with torch.no_grad():
            if CFG.use_amp:
                with torch.cuda.amp.autocast():
                    features = model.extract_feat(img)
            else:
                features = model.extract_feat(img)

        embeddings = features.detach().cpu().numpy().astype('float32')
        embeds.append(embeddings)

    del model
    embeddings = np.concatenate(embeds)
    del embeds
    gc.collect()
    return embeddings

## Prediction Function

In [ ]:
def get_predictions(df, cnn_embeddings_half, cnn_threshold=1.0, 
                    chunk=32, nearlest_one=True, max_preds=50):

    print('Finding similar ones...')
    CTS = len(df) // chunk
    if (len(df) % chunk) != 0:
        CTS += 1
        
    preds = []
    for j in tqdm(range(CTS)):
        a = j * chunk
        b = min((j+1) * chunk, len(df))
        cnn_cts = torch.matmul(cnn_embeddings_half, cnn_embeddings_half[a:b].T).T
        
        for k in range(b-a):
            sim = (cnn_cts[k,]  )
            sim_desc = torch.sort(sim, descending=True)
            
            IDX = sim_desc[1][sim_desc[0] > cnn_threshold][:max_preds].cpu().detach().numpy()
            o = df.iloc[IDX].posting_id.values
            
            # if (len(IDX) == 1) and nearlest_one:
            #     IDX = sim_desc[1][:2].cpu().detach().numpy()
            #     o = df.iloc[IDX].posting_id.values
            
            preds.append(o)
# cnn_cts, 
    del cnn_cts
    gc.collect()
    torch.cuda.empty_cache()
    
    return preds

# Calculating Predictions

In [ ]:
df, image_paths = read_dataset()

# Image Embeddings

In [ ]:
dataset = ShopeeDataset(df=df, img_dir=CFG.images_dir, transform=transform)
data_loader = DataLoader(dataset, batch_size=32, shuffle=False,
                         drop_last=False, pin_memory=True, num_workers=4)

In [ ]:
path = '../input/shopee/v45.pth'
checkpoint1 = torch.load(path, map_location=CFG.device)
params1['backbone'] = "resnet34"
params1['backbone'] = "deit_base_distilled_patch16_384"
backbone = timm.create_model(model_name=params1['backbone'], pretrained=False)
model1 = ShopeeNet(backbone, num_classes=0, fc_dim=params1['fc_dim'])
model1 = model1.to(CFG.device)
model1.load_state_dict(checkpoint1['model'], strict=False)
model1.train(False)
model1.p = params1['p_eval']

In [ ]:
cnn_embeddings = get_cnn_embeddings(model1, data_loader)
print('cnn_embeddings.shape:', cnn_embeddings.shape)

## Prediction

In [ ]:
cnn_embeddings_half = torch.tensor(cnn_embeddings, dtype=torch.float16).to(CFG.device)

predictions = get_predictions(df,
#                               F.normalize(cnn_embeddings_mean_half),
                              F.normalize(cnn_embeddings_half),
                              cnn_threshold=CFG.cnn_threshold,
                              chunk=CFG.chunk,
                              max_preds=CFG.max_preds,
                              nearlest_one=CFG.nearlest_one)

df['predictions'] = predictions

# Submission

In [ ]:
def combine_predictions(row):
    x = np.concatenate([row[col] for col in CFG.todo_predictions])
    return ' '.join( np.unique(x) )

In [ ]:
df['matches'] = df.apply(combine_predictions, axis=1)
df[['posting_id', 'matches']].to_csv('submission.csv', index=False)
submission_df = pd.read_csv('submission.csv')

In [ ]:
submission_df

## Compute CV